In [1]:
import pandas as pd
import re
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
from nltk.stem import WordNetLemmatizer
import warnings
warnings.filterwarnings("ignore")

In [2]:
df1=pd.read_csv(r"theatre.csv")
df2=pd.read_csv(r"theatre1.csv")
df=pd.concat([df1,df2])
df.reset_index(inplace=True)

In [3]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)
df

,index,movie,names,reviews
0,0,['Cinépolis - CRM Mall Shahdara',['Zuh','NEW' '' '' '' '' 'I watched Spiderman: Acros...
1,1,'Cinépolis - CRM Mall Shahdara',['Arif Niyazi','Heaven inside devil outside. Overall good. B...
2,2,'Cinépolis - CRM Mall Shahdara',['Shubham Tayal','It was ok but Cinepolis service was not that...
3,3,'Cinépolis - CRM Mall Shahdara',['Zeeshan Mohammad','3 months ago' 'Rating it 4 stars only becaus...
4,4,'Cinépolis - CRM Mall Shahdara',['Rahul Kumar',"'NEW' '' '' '' '' ""Worst movie experience. Th..."
...,...,...,...,...
2245,1095,'Ega Multiplex',['K.sakthi muthu','Newly renovated ega main screen with barco l...
2246,1096,'Ega Multiplex',['Shobana Kannan',"'NEW' ""We booked platinum but no use seats ar..."
2247,1097,'Ega Multiplex',['Vivek Raj','Good improvement. Except seats were not clea...
2248,1098,'Ega Multiplex',['Taha Muaaz','One of the oldest cinema theatres in city. V...


In [4]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords+=["new","more","month","ago","theatre","movie","like","theater",'pvr','imax','also','ag']
lemmatizer = WordNetLemmatizer()

In [5]:
def preprocessing(df):
    rev=[]
    stw=[]
    lm=[]
    for i in range(len(df)):
        df['movie'][i]=re.sub("\s+"," ",re.sub("\W"," ",df["movie"][i]))
        df['names'][i]=re.sub("\s+"," ",re.sub("\W"," ",df["names"][i]))
        df['reviews'][i]=re.sub("\s+"," ",re.sub("\W"," ",df["reviews"][i]))
        df['movie'][i]=df['movie'][i].lower()
        df['names'][i]=df['names'][i].lower()
        df['reviews'][i]=df['reviews'][i].lower()
        rev.append(df['reviews'][i].split(' '))
    for i in rev:
        new=[]
        for j in i:
            if j not in stopwords:
                 new.append(j)
        stw.append(new)
    for i in stw:
        new1=[]
        for j in i:
            new1.append(lemmatizer.lemmatize(j)) 
        lm.append(new1)
    return lm

In [6]:
prep=preprocessing(df)
df['Preprocessed']=prep
df

,index,movie,names,reviews,Preprocessed
0,0,cinépolis crm mall shahdara,zuh,new i watched spiderman across the spider ver...,"[, watched, spiderman, across, spider, verse, ..."
1,1,cinépolis crm mall shahdara,arif niyazi,heaven inside devil outside overall good but ...,"[, heaven, inside, devil, outside, overall, go..."
2,2,cinépolis crm mall shahdara,shubham tayal,it was ok but cinepolis service was not that ...,"[, ok, cinepolis, service, good, always, coz, ..."
3,3,cinépolis crm mall shahdara,zeeshan mohammad,3 months ago rating it 4 stars only because i...,"[, 3, month, rating, 4, star, went, watch, pre..."
4,4,cinépolis crm mall shahdara,rahul kumar,new worst movie experience the hall smells ve...,"[, worst, experience, hall, smell, foul, ac, w..."
...,...,...,...,...,...
2245,1095,ega multiplex,k sakthi muthu,newly renovated ega main screen with barco la...,"[, newly, renovated, ega, main, screen, barco,..."
2246,1096,ega multiplex,shobana kannan,new we booked platinum but no use seats are n...,"[, booked, platinum, use, seat, comfortable, l..."
2247,1097,ega multiplex,vivek raj,good improvement except seats were not clean ...,"[, good, improvement, except, seat, clean, guy..."
2248,1098,ega multiplex,taha muaaz,one of the oldest cinema theatres in city ver...,"[, one, oldest, cinema, theatre, city, close, ..."


In [10]:
d={}
for i in df['movie'].unique():
    words=[]
    for j in range(len(df)):
        if i==df['movie'][j]:
            for k in df['Preprocessed'][j]:
                words.append(k)
    d[i]=words

In [11]:
from collections import Counter
unq=[]
for i in d.values():
    c=Counter(i)
    if len(c)>5:
        l=6
    else:
        l=len(c)
    unq.append([list(filter(lambda x: d[x] == i, d))[0],c.most_common(10)[1:]])       

In [12]:
unq.pop(7)
unq.pop(11)
unq.pop(35)
unq.pop(39)

[' pvr naraina ',
 [('screen', 37),
  ('good', 22),
  ('experience', 20),
  ('mall', 20),
  ('one', 17),
  ('sound', 14),
  ('chennai', 14),
  ('place', 13),
  ('vr', 12)]]

In [13]:
import matplotlib.pyplot as plt
def graph():
    # Function to get top N keywords and frequencies from each cinema's data
    def get_top_keywords(cinema_data, N=5):
        keywords, frequencies = zip(*cinema_data)
        top_keywords = keywords[:N][::-1]
        top_frequencies = frequencies[:N][::-1]
        return top_keywords, top_frequencies

    # Create subplots
    num_cinemas = len(unq)
    num_rows = 9 # You can adjust the number of rows and columns as per your preference
    num_cols = (num_cinemas + num_rows - 1) // num_rows

    fig, axs = plt.subplots(num_rows, num_cols, figsize=(22, 30), sharex=True)

    for i, (cinema_name, keyword_data) in enumerate(unq):
        row = i // num_cols
        col = i % num_cols
        top_keywords, top_frequencies = get_top_keywords(keyword_data, N=5)

        axs[row, col].barh(range(len(top_keywords)), top_frequencies, alpha=0.7)
        axs[row, col].set_yticks(range(len(top_keywords)))
        axs[row, col].set_yticklabels(top_keywords)
        axs[row, col].set_xlabel('Frequency')
        axs[row, col].set_title(cinema_name)

    # Remove any empty subplots if there are any
    for i in range(num_cinemas, num_rows * num_cols):
        row = i // num_cols
        col = i % num_cols
        fig.delaxes(axs[row, col])

    # Adjust the layout
    plt.tight_layout()
    plt.subplots_adjust(top=0.9, right=0.85)  # Add space for the legend

    # Create a single stacked legend for all the subplots and place it outside the subplots
    cinema_names = [cinema_data[0] for cinema_data in unq]

    plt.suptitle('Top 5 Keywords for Each Cinema', fontsize=16)
    plt.show()

In [14]:
df.drop('index',axis=1)

,movie,names,reviews,Preprocessed
0,cinépolis crm mall shahdara,zuh,new i watched spiderman across the spider ver...,"[, watched, spiderman, across, spider, verse, ..."
1,cinépolis crm mall shahdara,arif niyazi,heaven inside devil outside overall good but ...,"[, heaven, inside, devil, outside, overall, go..."
2,cinépolis crm mall shahdara,shubham tayal,it was ok but cinepolis service was not that ...,"[, ok, cinepolis, service, good, always, coz, ..."
3,cinépolis crm mall shahdara,zeeshan mohammad,3 months ago rating it 4 stars only because i...,"[, 3, month, rating, 4, star, went, watch, pre..."
4,cinépolis crm mall shahdara,rahul kumar,new worst movie experience the hall smells ve...,"[, worst, experience, hall, smell, foul, ac, w..."
...,...,...,...,...
2245,ega multiplex,k sakthi muthu,newly renovated ega main screen with barco la...,"[, newly, renovated, ega, main, screen, barco,..."
2246,ega multiplex,shobana kannan,new we booked platinum but no use seats are n...,"[, booked, platinum, use, seat, comfortable, l..."
2247,ega multiplex,vivek raj,good improvement except seats were not clean ...,"[, good, improvement, except, seat, clean, guy..."
2248,ega multiplex,taha muaaz,one of the oldest cinema theatres in city ver...,"[, one, oldest, cinema, theatre, city, close, ..."


In [16]:
theaters_df=df
def calculate_similarity(user_input, theater_reviews):
    user_words = set(user_input.split())
    theater_words = set(theater_reviews.split())
    similarity = len(user_words.intersection(theater_words)) / len(user_words.union(theater_words))
    return similarity

def theaters(user_input, df, num_recommendations=3):
    theater_scores = []
    for index, theater_row in theaters_df.iterrows():
        similarity = calculate_similarity(user_input, theater_row['reviews'])
        theater_scores.append((index, similarity))
    theater_scores.sort(key=lambda x: x[1], reverse=True)
    recommendations = theater_scores[:num_recommendations]
    return recommendations

def Theater_Recommendation(user_input):
    #user_input = "I am looking for a theater with good sound and comfortable seating."
    recommended_theaters = theaters(user_input, theaters_df)
    print("Recommended Theaters:")
    for index, similarity in recommended_theaters:
        print(f"{theaters_df.at[index, 'movie']}")
#Theater_Recommendation("I am looking for a theater with good sound and comfortable seating.")